## Import picture(s) to OMERO server
- also see https://gitlab.com/openmicroscopy/incubator/omero-python-importer/-/blob/master/import.py

In [ ]:
import os
import json
import omero
import omero.clients
from omero.gateway import BlitzGateway
from omero.rtypes import rstring, rbool
from omero.model import ChecksumAlgorithmI
from omero.model.enums import ChecksumAlgorithmSHA1160
from omero.callbacks import CmdCallbackI

#### Server, Cedentials and Co

In [ ]:
server = " "
my_port = "4064"
username = " "
password = " "

#### Files & Dataset ID

In [ ]:
dataset_id = 8
files = ["too_1.png"]
files

### Set Up Connection

- create client

In [ ]:
c = omero.client(server)

- create Session

In [ ]:
c.createSession(username, password)

- set up connection

In [ ]:
conn = BlitzGateway(client_obj=c)

- test conncetion

In [ ]:
user = conn.getUser()
print("Current user:")
print("   ID:", user.getId())
print("   Username:", user.getName())
print("   Full Name:", user.getFullName())

### Create Fileset

In [ ]:
fileset = omero.model.FilesetI()
for f in files:
    entry = omero.model.FilesetEntryI()
    entry.setClientPath(rstring(f))
    fileset.addFilesetEntry(entry)
upload = omero.model.UploadJobI()
fileset.linkJob(upload)

### Create Settings

In [ ]:
settings = omero.grid.ImportSettings()
settings.doThumbnails = rbool(True)
settings.noStatsInfo = rbool(False)
settings.userSpecifiedTarget = None
settings.userSpecifiedName = None
settings.userSpecifiedDescription = None
settings.userSpecifiedAnnotationList = None
settings.userSpecifiedPixels = None
settings.checksumAlgorithm = ChecksumAlgorithmI()
s = rstring(ChecksumAlgorithmSHA1160)
settings.checksumAlgorithm.value = s

### Import

- set Managed Repository

In [ ]:
mrepo = c.getManagedRepository()

- Start Import

In [ ]:
proc = mrepo.importFileset(fileset, settings)

- Upload files to OMERO from local filesystem

In [ ]:
return_value = []
for i, fobj in enumerate(files):
    rfs = proc.getUploader(i)
    with open(fobj, 'rb') as f:
        print ('Uploading: : %s' % fobj)
        offset = 0
        block = []
        rfs.write(block, offset, len(block))  # Touch
        while True:
            block = f.read(1000 * 1000)
            if not block:
                break
            rfs.write(block, offset, len(block))
            offset += len(block)
        return_value.append(c.sha1(fobj))

In [ ]:
rfs.close()

In [ ]:
print ('Hashes:\n  %s' % '\n  '.join(return_value))

- Check upload (--> now image is visible under orphaned images)

In [ ]:
return_value[0]

In [ ]:
proc.verifyUpload(return_value)

In [ ]:
handle = proc.verifyUpload(return_value)

In [ ]:
callback = CmdCallbackI(c, handle)

In [ ]:
response = callback.getResponse()

- Close process to import fileset

In [ ]:
proc.close()

- Link image to dataset

In [ ]:
links = []
for p in response.pixels:
    print ('Imported Image ID: %d' % p.image.id.val)
    link = omero.model.DatasetImageLinkI()
    link.parent = omero.model.DatasetI(dataset_id, False)
    link.child = omero.model.ImageI(p.image.id.val, False)
    links.append(link)

In [ ]:
conn.getUpdateService().saveArray(links, conn.SERVICE_OPTS)

### Close Connection

In [ ]:
conn.close()

In [ ]:
c.closeSession()

In [ ]:
ret_val = []
for i, fobj in enumerate(files):
    rfs = proc.getUploader(i)
    with open(fobj, 'rb') as f:
        print ('Uploading: : %s' % fobj)
        offset = 0
        block = []
        rfs.write(block, offset, len(block))  # Touch
        while True:
            block = f.read(1000 * 1000)
            if not block:
                break
            rfs.write(block, offset, len(block))
            offset += len(block)
        ret_val.append(c.sha1(fobj))

In [ ]:
rfs.close()

In [ ]:
ret_val

In [ ]:
handle = proc.verifyUpload(ret_val)
handle

In [ ]:
cb = CmdCallbackI(c, handle)
cb

In [ ]:
rsp_2 = cb.getResponse()

In [ ]:
dataset_id = 6
links = []
for p in rsp_2.pixels:
    print ('Imported Image ID: %d' % p.image.id.val)
    link = omero.model.DatasetImageLinkI()
    link.parent = omero.model.DatasetI(dataset_id, False)
    link.child = omero.model.ImageI(p.image.id.val, False)
    links.append(link)


In [ ]:
conn.getUpdateService().saveArray(links, conn.SERVICE_OPTS)

#### close connection

In [ ]:
conn.close()

In [ ]:
c.closeSession()

In [ ]:
def startUpload(self, files):
     mrepo = self.client.getManagedRepository()
     fs = FilesetI()
     fs.linkJob(UploadJobI())
     for file in files:
         entry = FilesetEntryI()
         entry.clientPath = rstring("%s/%s" % (
             file.path.val, file.name.val
         ))
         fs.addFilesetEntry(entry)
     settings = ImportSettings()
     settings.checksumAlgorithm = ChecksumAlgorithmI()
     settings.checksumAlgorithm.value = rstring("SHA1-160")
     return mrepo.importFileset(fs, settings)